In [1]:
import pandas as pd
import numpy as np
import util
import keras as K
from keras.optimizers import Adam
# import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

d:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
phase2_ground_truth = pd.read_hdf('data/phase2_ground_truth.hdf')
phase2_train = pd.read_csv('data/phase2_train.csv')
trainData = pd.read_csv('data/train.csv')
testData = pd.read_csv('data/test.csv')

In [3]:
trainData = phase2_train
trainData['timestamp'] = pd.to_datetime(trainData['timestamp'], unit='s')

In [4]:
data = trainData[trainData['KPI ID']=='42d6616d-c9c5-370a-a8ba-17ead74f3114']
data = data.set_index('timestamp')
data = data.drop(columns=['KPI ID'])
print(data[data['label']==1].count()/data.count())

value    0.00962
label    0.00962
dtype: float64


In [5]:
data = preprocessing.minmax_scale(data)

In [6]:
data

array([[0.27259317, 0.        ],
       [0.27694099, 0.        ],
       [0.26661491, 0.        ],
       ...,
       [0.28897516, 0.        ],
       [0.28253106, 0.        ],
       [0.29029503, 0.        ]])

In [7]:
trainInput,trainOutput = util.splitInputOutput(data,sequence_length=30,feature_column_start=0,feature_column_end=-1)

In [8]:
np.array(trainInput).shape

(146223, 30, 1)

In [9]:
np.array(trainOutput).shape

(146223,)

In [10]:
'''
暴力重复过采样至正负1:1，不可取，只做示意
'''
idxRecord = []
for idx in range(len(trainOutput)):
    if trainOutput[idx] == 1:
        idxRecord.append(idx)
overSampleTimes = int(len(trainInput)/len(idxRecord) -2)
print('oversample times: ', overSampleTimes)
for i in range(overSampleTimes):
    for idx in idxRecord:
        trainInput.append(trainInput[idx])
        trainOutput.append(trainOutput[idx])
PositiveCount = 0
for idx in range(len(trainOutput)):
    if trainOutput[idx] == 1:
        PositiveCount += 1
print('Positive sample ratio:',PositiveCount/len(trainInput))

oversample times:  101
Positive sample ratio: 0.49774217042971597


In [11]:
K.backend.clear_session()
opt = Adam(lr=1e-3)
model = util.buildModel(input_shape=(None,1),drop=0.5,opt=opt,loss='mse')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, None, 128)         33792     
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, None, 256)         263168    
_________________________________________________________________
dropout_2 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               164352    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 256)               0         
__________

In [12]:
trainX = np.array(trainInput)
trainY = np.array(trainOutput)
print(trainX.shape)
print(trainY.shape)

(288330, 30, 1)
(288330,)


In [13]:
trainX = K.utils.normalize(trainX)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(trainX, trainY, test_size=0.2, random_state=123)

In [15]:
model.fit(x=X_train,y=y_train,batch_size=256,epochs=100,validation_data=(X_test,y_test))

Train on 230664 samples, validate on 57666 samples
Epoch 1/100
  7936/230664 [>.............................] - ETA: 6:28 - loss: 0.2504

KeyboardInterrupt: 